In [2]:
!pip install pytorch-transformers
!git clone https://github.com/BMaksim/classifocation-with-BERT

     |████████████████████████████████| 184kB 30.0MB/s 
     |████████████████████████████████| 1.0MB 39.4MB/s 
     |████████████████████████████████| 870kB 31.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=200847aee72a821410cd4ede3e0eef39c976d89a94f701dbcc63097fd9b95aff
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
Cloning into 'classifocation-with-BERT'...
remote: Enumerating objects: 2432, done.
remote: Counting objects: 100% (2432/2432), done.
remote: Compressing objects: 100% (1565/1565), done.
remote: Total 2432 (delta 826), reused 2405 (delta 810), pack-reused 0
Receiving objects: 100% (2432/2432), 10.35 MiB | 24.38 MiB/s, done.
Resolving deltas: 100% (826/826), done.


In [2]:
!ls classifocation-with-BERT/

back  data  outputs  __pycache__  README.md  Solution.ipynb  text_to_tsv.py


In [0]:
import torch
import numpy as np
from pytorch_transformers import BertTokenizer, BertConfig
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from pytorch_transformers import AdamW, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
# Извлечение данных из txt и приведение к необходимому формату
import re
import pandas as pd



def txt_to_lists (dir):
    handle = open(dir, "r")
    datal = handle.readlines() 
    data = ""
    for one in datal:
        data += one
    data = re.split("\t|\n", data)
    data = data[:-1]
    labels, examples = [], []
    i = 0
    for one in data:
        if i%2 == 0:
            if one == "No":
                labels.append(0)
            else:
                labels.append(1)
        else:
            examples.append("[CLS] " + one + " [SEP]")
        i += 1
    handle.close()
    return (labels, examples)


train_labels, train_examples = txt_to_lists("classifocation-with-BERT/data/trainSet")
test_labells, test_examples = txt_to_lists("classifocation-with-BERT/data/testSet")

In [29]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case = True)
tokenized_data = [tokenizer.tokenize(sentence) for sentence in train_examples]
input_ids = [tokenizer.convert_tokens_to_ids(token) for token in tokenized_data]
input_ids = pad_sequences(
    input_ids,
    maxlen=128,
    dtype="long",
    truncating="post",
    padding="post"
)

train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(train_labels)
print(train_labels)
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader( train_data, sampler = RandomSampler(train_data), batch_size = 32)

tensor([0, 0, 0,  ..., 1, 1, 1])


In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

100%|██████████| 440473133/440473133 [00:05<00:00, 74214841.85B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [0]:
model.train()

for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_labels = batch
    loss = model(b_input_ids, token_type_ids=None, labels=b_labels)

    optimizer.zero_grad()
    loss[0].backward()
    optimizer.step()

In [28]:
tokenized_data = [tokenizer.tokenize(sentence) for sentence in test_examples]
input_ids = [tokenizer.convert_tokens_to_ids(token) for token in tokenized_data]
input_ids = pad_sequences(
    input_ids,
    maxlen=128,
    dtype="long",
    truncating="post",
    padding="post"
)

prediction_inputs = torch.tensor(input_ids)
prediction_labels = torch.tensor(test_labells)

prediction_data = TensorDataset(prediction_inputs, prediction_labels)

prediction_dataloader = DataLoader(prediction_data, sampler=SequentialSampler(prediction_data), batch_size=32)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [0]:
model.eval()
test_preds = []

for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_labels = batch

    with torch.no_grad():
        logits = model(b_input_ids, token_type_ids=None)

    logits = logits[0].detach().cpu().numpy()

    batch_preds = np.argmax(logits, axis=1)
    test_preds.extend(batch_preds)

In [44]:
def report(labels, preds):
    accuracy = accuracy_score(labels, preds)
    recall = recall_score(labels, preds, average="binary")
    f1 = f1_score(labels, preds, average="binary")
    precision = precision_score(labels, preds, average="binary")
    return {"accuracy": accuracy, "recall": recall, "f1": f1, "precision": precision}

result = report(test_labells, test_preds)
print(result)

{'accuracy': 0.8477822580645161, 'recall': 0.8478964401294499, 'f1': 0.7762962962962963, 'precision': 0.7158469945355191}
